In [7]:
import pandas as pd

ms = pd.read_csv("measurement_db-Measurement.csv")
ms

,id,batchId,xRotation,yRotation,zRotation,xAcceleration,yAcceleration,zAcceleration
0,1,1,-0.026231,0.049400,0.030093,0.223878,0.124726,9.260074
1,2,1,-0.029028,-0.008522,0.020506,0.389133,0.246909,9.851700
2,3,1,0.024900,-0.001598,0.004927,0.389133,0.246909,9.851700
3,4,1,-0.020373,-0.000533,0.003595,0.355633,-0.119641,9.767951
4,5,1,0.015446,-0.006658,0.000399,0.410818,-0.033350,9.719945
...,...,...,...,...,...,...,...,...
2195,2196,22,0.004261,-0.012250,-0.236615,0.279063,0.084048,9.731909
2196,2197,22,0.004261,-0.012250,-0.236615,0.408425,0.443420,9.595368
2197,2198,22,0.004261,-0.012250,-0.236615,0.317348,0.333201,9.710373
2198,2199,22,0.004261,-0.012250,-0.236615,0.245414,0.112762,9.935597


In [8]:
bt = pd.read_csv("measurement_db-Batch.csv")
bt

,id,type
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,1


In [208]:
import numpy as np
data = pd.DataFrame(columns=['type', 'sensors'])
for i in set(ms['batchId'].tolist()):
    allMeasurements = ms.loc[ms['batchId'] == i]
    filtered = allMeasurements.drop('id', inplace=False, axis=1).drop('batchId', inplace=False, axis=1).to_numpy()
    #print(allMeasurements[0])
    flat = filtered.flatten()
    batchType = bt.loc[bt['id'] == i].iat[0,1]
    data.loc[len(data)] = {
    "type": batchType,
    "sensors": flat
    }
data

,type,sensors
0,1,"[-0.026231363415718, 0.0494001805782318, 0.030..."
1,1,"[0.000932078924961388, 0.0007989247678779066, ..."
2,1,"[0.0046603945083916, -0.000932078924961388, 0...."
3,1,"[0.0647129043936729, 0.0189078859984874, 0.009..."
4,1,"[-0.0145137999206781, -0.0077229393646121, -0...."
5,1,"[-0.0246335137635469, -0.0030625448562204, -0...."
6,1,"[0.0005326165119186044, -0.0213046595454216, 0..."
7,1,"[-0.0062582441605627, 0.0050598569214344, -0.0..."
8,1,"[0.0563241951167583, 0.1213034093379974, -0.06..."
9,1,"[0.0370168462395668, -0.4901403486728668, 0.13..."


In [269]:
X = np.stack(data['sensors'].values)
y = data['type'].values

In [270]:
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(data["sensors"].to_numpy(), data["type"].to_numpy(), test_size=0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(data["sensors"], data["type"], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_test)
print(y_test)

[[-2.62313634e-02  4.94001806e-02  3.00928336e-02 ...  4.17996198e-01
   5.94317317e-01  9.87084198e+00]
 [ 3.49396437e-01 -1.26682842e+00  3.32006502e+00 ...  4.03639257e-01
   3.81056994e-01  9.73908710e+00]
 [ 5.63241951e-02  1.21303409e-01 -6.00525104e-02 ... -2.99805713e+00
   9.27218795e-01  1.77664089e+01]
 [ 9.32078925e-04  7.98924768e-04  2.39677425e-03 ...  4.53888535e-01
   2.22981155e-01  9.99302578e+00]
 [-2.06921518e-01 -6.13840520e-02  3.99196088e-01 ...  1.06465650e+00
  -8.50199819e-01  9.31989479e+00]]
[1 2 1 1 2]


In [304]:
from keras.models import Sequential
from keras.layers import Dense

import tensorflow as tf

#X_train = np.asarray(X_train.array).astype('float32')

# define the model
model = Sequential()
model.add(Dense(10, input_dim=600, activation='relu'))
model.add(Dense(1, activation='linear'))

# compile the model
model.compile(loss='mean_squared_error', metrics=['accuracy'] ,optimizer='adam')

# train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50
1/1 [==============================] - 1s 641ms/step - loss: 3.9382 - accuracy: 0.2667 - val_loss: 4.8718 - val_accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 27ms/step - loss: 9.4013 - accuracy: 0.5333 - val_loss: 0.2593 - val_accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 29ms/step - loss: 2.5738 - accuracy: 0.3333 - val_loss: 3.5061 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 33ms/step - loss: 2.7226 - accuracy: 0.2667 - val_loss: 7.3108 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 36ms/step - loss: 4.5219 - accuracy: 0.2000 - val_loss: 5.9106 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 41ms/step - loss: 3.0329 - accuracy: 0.2000 - val_loss: 2.5026 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 33ms/step - loss: 0.9731 - accuracy: 0.3333 - val_loss: 0.4125 - val_accuracy: 0.0000e+00
Epoch 

In [306]:
loss, accuracy = model.evaluate(X_test, y_test)


1/1 [==============================] - 0s 23ms/step - loss: 14.8831 - accuracy: 0.6000
